In [ ]:
%pip install -r requirements.txt

In [1]:
path_in =r"C:\Users\Tony_Mony\Desktop\projects\etl_project\data\in.xlsx"

In [3]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string
from map import INDICATOR_MAPPING,REGION_NORMALIZATION
# Ваш существующий INDICATOR_MAPPING остается без изменений

def normalize_region(region_name):
    """Приводит название региона к стандартному виду"""
    if not region_name or not isinstance(region_name, str):
        return None
    
    # Приводим к нижнему регистру и удаляем лишние символы
    cleaned = re.sub(r'[^\w\s\-]', '', region_name.lower().strip())
    
    # Ищем совпадения с нашим словарем
    for pattern, standard in REGION_NORMALIZATION.items():
        if re.search(pattern, cleaned):
            return standard
    
    # Если не нашли совпадение, возвращаем оригинал с базовой очисткой
    return region_name.strip()
def extract_company(text):
    # Нормализация текста
    text = re.sub(r'_{2,}', ' ', text)  # Заменяем множественные подчеркивания
    text = re.sub(r'\s+', ' ', text)  # Заменяем множественные пробелы
    text = text.replace('«', '"').replace('»', '"')  # Стандартизируем кавычки
    
    # Улучшенный паттерн поиска
    pattern = r'''
        (ПАО|АО|ООО|ЗАО|НКО)  # Тип организации
        \s*                    # Возможные пробелы
        "                      # Открывающая кавычка
        (                      # Начинаем захват названия
          (?:                  # Группа без захвата
            [^"\n]+            # Любые символы кроме кавычки и переноса строки
            (?:"[^"\n]+)*      # Вложенные кавычки с текстом
          )                    #
        )                      # Конец названия
        "                      # Закрывающая кавычка
        (?!\S)                 # Не должно быть букв/цифр после
    '''
    
    match = re.search(pattern, text, re.VERBOSE | re.IGNORECASE)
    if match:
        company_type = match.group(1).upper()
        company_name = match.group(2)
        
        # Очистка названия (без обрезания существенной части)
        clean_name = company_name.strip(' _-')
        return f'{company_type} "{clean_name}"'
    
    # Дополнительный поиск для случаев с большим количеством пробелов
    alt_pattern = r'(ПАО|АО|ООО|ЗАО|НКО)\s+"([^"]+)"'
    alt_match = re.search(alt_pattern, text, re.IGNORECASE)
    if alt_match:
        return f'{alt_match.group(1).upper()} "{alt_match.group(2).strip()}"'
    
    return None
def cut_company_name(company):
    company = company.split()
    if len(company)>=3:
        company=company[:3]
    return " ".join(company)

def extract_metadata(sheet):
    """Извлекает метаданные из листа"""
    metadata = {
        'subject':None,
        'company': None,
        'license': None,
        'area': None,
        'vink': None
    }
    
    # Поиск названия компании
    for row in sheet.iter_rows(values_only=True):
        if not any(row):
            continue
        
        row_text = ' '.join(str(cell) for cell in row if cell).replace('\n', ' ')
        
        if "(наименование компании)" in row_text or "выполненных в" in row_text :
            match = extract_company(row_text)
            if match:
                metadata['company'] = cut_company_name(match)
            break
    for row in sheet.iter_rows(max_row=15, values_only=True):
        if not any(row):
            continue
            
        row_text = ' '.join(str(cell) for cell in row if cell)
        if not row_text:
            continue
            
        # Проверяем типичные упоминания региона
        if any(word in row_text.lower() for word in ['хмао', 'ханты', 'югра', 'ямал', 'янао']):
            metadata['subject'] = normalize_region(row_text)
            break
    # Поиск лицензии и ВИНК
    license_pattern = re.compile(r'Лицензия\s*([А-Я]{2,}\s*\d+\s*[А-Я]{2,})[\s,]*([А-Яа-я\-\s]+)')
    vink_pattern = re.compile(
    r'ВИНК\*\s*((?:ПАО|АО|ООО|ЗАО|НКО)\s*)?'  # Орг. форма (опционально, сохраняем)
    r'("[^"]*(?:"[^"]*)*"|[^"\s]+)'  # Название (с кавычками или без)
    r'(?=\s*(?:доля|$|\n))',  # Стоп-символы
    flags=re.IGNORECASE
)
      
    for row in sheet.iter_rows(values_only=True):
        if not any(row):
            continue
        row_text = ' '.join(str(cell) for cell in row if cell)
        
        # Поиск лицензии
        license_match = license_pattern.search(row_text)
        if license_match and not metadata['license']:
            metadata['license'] = license_match.group(1).strip()
            metadata['area'] = license_match.group(2).strip().replace(" Шельфовое продолжение",'')
        
        # Поиск ВИНК
        vink_match = vink_pattern.search(row_text)
        if vink_match:
            # Объединяем все непустые группы через пробел
            combined = ' '.join(group for group in vink_match.groups() if group)
            metadata['vink'] = combined
    
    return metadata

def process_workbook(file_path):
    wb = load_workbook(file_path,data_only=True)
    all_data = []
    i=1
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        
        # Извлекаем метаданные один раз для листа
        metadata = extract_metadata(sheet)
        
        # Находим колонки с данными
        fact_col = plan_col = None
        for row in sheet.iter_rows(max_row=10):
            for cell in row:
                if cell.value and isinstance(cell.value, str):
                    text = str(cell.value).lower()
                    if "2022" in text and "факт" in text:
                        fact_col = cell.column_letter
                    elif "2023" in text and "план" in text:
                        plan_col = cell.column_letter
        
        if not fact_col or not plan_col:
            print(f"Не найдены колонки с данными в листе: {sheet_name}")
            continue
        
        # Сначала собираем ВСЕ показатели за 2022 год
        records_2022 = {
            'ID':i,
            'Субъект РФ': metadata.get("subject"),
            'Период (год)': 2022,
            'план/факт':'факт',
            'Главная компания': metadata.get('company'),
            'Недропользователь': metadata.get('vink'),
            '№ лицензии': metadata.get('license'),
            'участок': metadata.get('area'),
            'Оценка выполнения лицензионных условий': None
        }
        i+=1
        
        # Затем ВСЕ показатели за 2023 год
        records_2023 = {
            'ID':i,
            'Субъект РФ': metadata.get("subject"),
            'Период (год)': 2023,
            'план/факт': 'план',
            'Главная компания': metadata.get('company'),
            'Недропользователь': metadata.get('vink'),
            '№ лицензии': metadata.get('license'),
            'участок': metadata.get('area'),
            'Оценка выполнения лицензионных условий': None
        }
        i+=1
        # Заполняем показатели
        for row in sheet.iter_rows(min_row=2, values_only=True):
            if not row or not isinstance(row[0], str):
                continue
                
            p_p = row[0].strip()
            if p_p in INDICATOR_MAPPING:
                col_name = INDICATOR_MAPPING[p_p]
                records_2022[col_name] = row[column_index_from_string(fact_col)-1]
                records_2023[col_name] = row[column_index_from_string(plan_col)-1]
        
        # Добавляем в общий список
        all_data.append(records_2022)
        all_data.append(records_2023)
    
    # Создаем DataFrame
    df = pd.DataFrame(all_data)
    
    # Заполняем None для отсутствующих показателей
    for col in INDICATOR_MAPPING.values():
        if col not in df.columns:
            df[col] = None
    df.iloc[:, 8] = df.iloc[:, 72]

# Удаление колонки 'BU'
    df.drop(df.columns[72], axis=1, inplace=True)
    return df


# Сохранение в Excel и CSV
def save_results(df, base_path):
    # Сохранение в Excel
    excel_path = f"{base_path}/результат.xlsx"
    df.to_excel(excel_path, index=False)
    
    # Сохранение в CSV
    csv_path = f"{base_path}/результат.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')


In [4]:
#запуск ф-ий
out_path =r'C:\Users\Tony_Mony\Desktop\projects\etl_project\results' 
result_df = process_workbook(path_in)
save_results(result_df,out_path)    


In [5]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from map import SHORT_NAMES,DB_COL

DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'dbname': 'postgres',
    'user': "postgres",
    'password': "postgres"
}

# Создаем подключение
engine = create_engine(f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}")
def generate_create_table_sql():
    """Генерирует SQL для создания таблицы с короткими именами колонок"""
    columns = []
    
    # Базовые колонки
    base_columns = [
        'id SERIAL PRIMARY KEY',
        'subject VARCHAR(100)',
        'company VARCHAR(200)',
        'license_number VARCHAR(50)',
        'area VARCHAR(200)',
        'vink VARCHAR(200)',
        'year INTEGER',
        'plan_fact VARCHAR(10)',
        'license_evaluation FLOAT'
    ]
    
    # Создаем словарь для маппинга оригинальных названий в короткие

    
    # Добавляем колонки индикаторов
    indicator_columns = [f"{SHORT_NAMES[key]} FLOAT" for key in INDICATOR_MAPPING.keys()]
    
    return f"CREATE TABLE IF NOT EXISTS oil_gas_data ({', '.join(base_columns + indicator_columns)})"

def load_to_postgres(df):
    """Загружает данные в PostgreSQL с короткими именами колонок"""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Создаем таблицу
    cur.execute(generate_create_table_sql())
    
    # Полный маппинг оригинальных названий в короткие
    column_mapping = {
        # Базовые колонки
        'ID': 'id',
        'Субъект РФ': 'subject',
        'Главная компания': 'company',
        'Недропользователь': 'vink',
        '№ лицензии': 'license_number',
        'участок': 'area',
        'Период (год)': 'year',
        'план/факт': 'plan_fact',
        'Оценка выполнения лицензионных условий': 'license_evaluation',
        
        # Индикаторы - автоматически генерируем на основе INDICATOR_MAPPING
        **{v: k for k, v in DB_COL.items()}
    }
    
    
   
    
    # Подготавливаем SQL для вставки (только существующие колонки)
    available_columns = [col for col in df.columns if col in column_mapping]
    placeholders = ', '.join(['%s'] * len(available_columns))
    sql = f"INSERT INTO oil_gas_data ({', '.join([column_mapping[col] for col in available_columns])}) VALUES ({placeholders})"
    
    # Загружаем данные построчно
    for _, row in df.iterrows():
        values = [row[col] if pd.notna(row[col]) else None for col in available_columns]
        cur.execute(sql, values)
    
    conn.commit()
    print(f"Успешно загружено {len(df)} записей")
    cur.close()
    conn.close()



In [6]:
load_to_postgres(result_df)

Успешно загружено 20 записей


результат заливки 
![alt text](../results/image.png)
